In [ ]:
!pip install numpy matplotlib

In [48]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [60]:
# inserisci la funzione di input

func_str = input("Inserisci la funzione in x (es: x**2 + 3): ")


def f(x):
    try:
        return eval(func_str, {"x": x, "math": math, "__builtins__": {}})
    except Exception as e:
        print(f"Errore nell'esecuzione della funzione: {e}")
        return 0


In [66]:
# inserisci l'intervallo della funzione e il numero di punti casuali da generare

try:
    a = float(input("Estremo inferiore dell'intervallo: "))
    b = float(input("Estremo superiore dell'intervallo: "))
    n_punti = int(input("Numero di punti casuali da generare: "))
except ValueError:
    print("Errore: inserisci numeri validi.")

if a >= b or n_punti <= 0:
    print("Errore: intervallo o numero di punti non valido.")
        

In [67]:
# trova il massimo della funzione
x_density = np.linspace(a, b, 10000) # calcoliamo 1000 valori equidistanti sulle x
y_density = [f(x) for x in x_density]

y_min = min(0, min(y_density))  # incluso lo 0 per gestire funzioni negative
y_max = max(0, max(y_density))

In [68]:
# montecarlo
count_under = 0

# array di salvataggio dei punti
x_under, y_under = [], []
x_over, y_over = [], []

for _ in range(n_punti):
    x_rand = a + (b - a) * np.random.random()
    y_rand = y_min + (y_max - y_min) * np.random.random()
    y_func = f(x_rand)

    if (y_rand >= 0 and y_rand <= y_func) or (y_rand < 0 and y_rand >= y_func):
        count_under += 1
        x_under.append(x_rand)
        y_under.append(y_rand)
    else:
        x_over.append(x_rand)
        y_over.append(y_rand)

In [69]:
# risultato area
area_rect = (b - a) * (y_max - y_min)
integral = area_rect * (count_under / n_punti)

print(f"\nStima dell'integrale geometrico di {func_str} da {a} a {b} con {n_punti} punti: {integral:.6f}")


Stima dell'integrale geometrico di math.sin(x) da 0.0 a 3.14 con 1000000 punti: 2.000883


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x_density, y_density, label="Funzione", color="blue")
plt.axhline(0, color="black", linewidth=0.5)
plt.scatter(x_under, y_under, color="green", s=10, alpha=0.5, label="Punti sotto la curva")
plt.scatter(x_over, y_over, color="red", s=10, alpha=0.5, label="Punti sopra la curva")
plt.title("Montecarlo Geometrico")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()